In [33]:
import pandas as pd
df= pd.read_csv('회귀모델_전처리완료_v1.0.csv')

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1161 entries, 0 to 1160
Data columns (total 13 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Unnamed: 0                           1161 non-null   int64  
 1   브랜드                                  1161 non-null   object 
 2   유통 업체                                1161 non-null   object 
 3   카테고리                                 1161 non-null   object 
 4   포장 형태                                1161 non-null   object 
 5   최근 3년간 광고모델 및 IP 협업 여부 (2022~2024년)  1161 non-null   int64  
 6   100ml당 가격                            1161 non-null   int64  
 7   용량                                   1161 non-null   int64  
 8   대체당                                  1161 non-null   object 
 9   음료명 기준 제로 여부                         1161 non-null   object 
 10  인기도                                  1161 non-null   float64
 11  대체당 여부                        

In [35]:
# '대체당' 열 삭제
df= df.drop(['Unnamed: 0','브랜드','대체당','대체당 여부'], axis=1)

In [36]:
df

,유통 업체,카테고리,포장 형태,최근 3년간 광고모델 및 IP 협업 여부 (2022~2024년),100ml당 가격,용량,음료명 기준 제로 여부,인기도,대체당 조합
0,광동제약,차,페트,1,322,200,일반 음료,69.654006,대체당 없음
1,광동제약,차,페트,1,361,200,일반 음료,43.246780,대체당 없음
2,광동제약,차,유리병,0,528,150,일반 음료,73.306406,대체당 없음
3,광동제약,차,페트,0,273,500,일반 음료,40.011834,대체당 없음
4,광동제약,차,페트,0,296,500,일반 음료,58.161078,대체당 없음
...,...,...,...,...,...,...,...,...,...
1156,서울우유협동조합,과·채음료,페트,0,533,210,일반 음료,41.445982,대체당 없음
1157,서울우유협동조합,과·채음료,페트,0,510,210,일반 음료,62.688344,대체당 없음
1158,서울우유협동조합,유제품/대체유,팩,0,354,190,일반 음료,55.055714,대체당 없음
1159,서울우유협동조합,유제품/대체유,파우치,0,217,190,일반 음료,46.012262,대체당 없음


- 컬럼명 변경: 편의를 위해 컬럼명을 변경합니다.

In [37]:
column_mapper = {
    '유통 업체': 'distributor',
    '카테고리': 'category',
    '포장 형태': 'packaging',
    '최근 3년간 광고모델 및 IP 협업 여부 (2022~2024년)': 'ad_collab',
    '100ml당 가격': 'price_100ml',
    '용량': 'volume',
    '음료명 기준 제로 여부': 'zero_type',
    '인기도': 'popularity',
    '대체당 조합': 'sweetener_mix'
}

df = df.rename(columns=column_mapper)

In [29]:
df = df.head()

In [38]:
df.describe(include='object')

,distributor,category,packaging,zero_type,sweetener_mix
count,1161,1161,1161,1161,1161
unique,18,8,7,2,4
top,롯데칠성음료,탄산음료,페트,일반 음료,대체당 없음
freq,173,250,522,956,695


In [38]:
df.describe()

,ad_collab,price_100ml,volume,popularity
count,1161.000000,1161.000000,1161.000000,1161.000000
mean,0.403962,312.901809,443.474591,46.504633
std,0.490902,195.924833,392.140584,23.553922
min,0.000000,53.000000,75.000000,0.000000
25%,0.000000,174.000000,200.000000,30.861535
50%,0.000000,253.000000,300.000000,46.263781
75%,1.000000,398.000000,500.000000,62.208756
max,1.000000,1793.000000,3000.000000,100.000000


In [45]:
df['sweetener_mix'].value_counts()

sweetener_mix
대체당 없음       695
합성 대체당       189
천연+합성 대체당    161
천연 대체당       116
Name: count, dtype: int64

CatBoost는 자체적으로 범주형 변수를 처리하므로 별도의 인코딩이 필요 없습니다.
연속형 변수는 별도의 스케일링 없이 원본 그대로 사용 가능합니다.

In [10]:
!pip install catboost

You should consider upgrading via the 'C:\Users\user\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


## 데이터 요약
- 독립
    - 범주: 유통업체(16), 카테고리(8), 포장형태(7), IP 협업 (2), 음료명 기준 제로 여부(3), 대체당 조합(4)
    - 연속: 100ml당 가격, 용량
- 종속
    - 인기도

## 모델 선택
- 현재 데이터는 범주형 변수가 연속형 변수보다 많다. 따라서 범주형 변수 처리가 용이한 알고리즘이 적절하다.

## 모델 후보
- Catboost
- Random Forest
- LightGBM

In [10]:

import numpy as np
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# 1. 필요한 컬럼 선택
feature_columns = [
   'distributor', 'category', 'packaging', 'zero_type', 'sweetener_mix',  # 범주형
   'ad_collab', 'price_100ml', 'volume'  # 연속형
]
target_column = 'popularity'

X = df[feature_columns]
y = df[target_column]

# 2. 범주형 변수 리스트 생성
categorical_features = [
   'distributor',
   'category',
   'packaging',
   'zero_type',
   'sweetener_mix'
]


# 4. 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
   X, y,
   test_size=0.2,
   random_state=42
)

# 5. CatBoost 모델 정의
cat_model = CatBoostRegressor(
   iterations=1000,
   learning_rate=0.1,
   depth=6,
   cat_features=categorical_features,
   eval_metric='RMSE',
   random_seed=42,
   verbose=200
)

# 6. 모델 학습
cat_model.fit(
   X_train, y_train,
   eval_set=(X_test, y_test),
   early_stopping_rounds=100,
   verbose=200
)

# 7. 예측 및 평가지표 계산
y_pred_train = cat_model.predict(X_train)
y_pred_test = cat_model.predict(X_test)

# 훈련 세트 평가
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
train_r2 = r2_score(y_train, y_pred_train)

# 테스트 세트 평가
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
test_r2 = r2_score(y_test, y_pred_test)

print("\n=== 모델 성능 평가 ===")
print("\n훈련 세트 성능:")
print(f"RMSE: {train_rmse:.4f}")
print(f"R2 Score: {train_r2:.4f}")

print("\n테스트 세트 성능:")
print(f"RMSE: {test_rmse:.4f}")
print(f"R2 Score: {test_r2:.4f}")

# 8. 특성 중요도 확인
feature_importance = pd.DataFrame({
   'feature': feature_columns,
   'importance': cat_model.feature_importances_
})
feature_importance = feature_importance.sort_values('importance', ascending=False)
print("\n=== 특성 중요도 ===")
print(feature_importance)

0:	learn: 23.2003384	test: 22.4758750	best: 22.4758750 (0)	total: 154ms	remaining: 2m 33s
200:	learn: 14.1958843	test: 18.5424188	best: 18.5424188 (200)	total: 2.37s	remaining: 9.42s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 18.51625796
bestIteration = 202

Shrink model to first 203 iterations.

=== 모델 성능 평가 ===

훈련 세트 성능:
RMSE: 15.8976
R2 Score: 0.5500

테스트 세트 성능:
RMSE: 18.5163
R2 Score: 0.3461

=== 특성 중요도 ===
         feature  importance
0    distributor   35.448179
1       category   19.610642
6    price_100ml   11.822751
4  sweetener_mix    9.974611
2      packaging    9.486537
7         volume    6.045326
5      ad_collab    5.809024
3      zero_type    1.802930


In [11]:
df['popularity'].describe()

count    1161.000000
mean       46.504633
std        23.553922
min         0.000000
25%        30.861535
50%        46.263781
75%        62.208756
max       100.000000
Name: popularity, dtype: float64

이는 100점 만점의 시험에서 평균적으로 18.5점 정도 틀릴 수 있다고 볼 수 있다.

In [13]:
from category_encoders import TargetEncoder

In [17]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 8.2 MB/s eta 0:00:00


In [15]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from category_encoders import TargetEncoder

# 1. 데이터 준비
feature_columns = [
    'distributor', 'category', 'packaging', 'zero_type', 'sweetener_mix',  # 범주형
    'ad_collab', 'price_100ml', 'volume'  # 연속형
]
categorical_features = [
    'distributor', 'category', 'packaging', 'zero_type', 'sweetener_mix'
]
target_column = 'popularity'

X = df[feature_columns]
y = df[target_column]

# 2. 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)



# Target Encoding
te = TargetEncoder()
X_train_rf = X_train.copy()
X_test_rf = X_test.copy()
X_train_rf[categorical_features] = te.fit_transform(X_train[categorical_features], y_train)
X_test_rf[categorical_features] = te.transform(X_test[categorical_features])

In [16]:
X_train_rf


,distributor,category,packaging,zero_type,sweetener_mix,ad_collab,price_100ml,volume
759,44.911086,40.361693,43.882723,52.353464,45.419530,0,187,500
816,44.911086,44.178008,43.882723,45.516546,46.464956,1,98,1000
862,52.193048,55.236580,52.753494,45.516546,48.158371,0,255,190
394,52.112058,44.178008,43.882723,45.516546,46.464956,0,164,500
693,44.911086,40.361693,43.882723,45.516546,46.464956,0,177,500
...,...,...,...,...,...,...,...,...
1044,40.032194,42.353763,43.882723,45.516546,47.663541,1,298,235
1095,40.114500,40.361693,43.882723,45.516546,46.464956,0,318,300
1130,43.934227,42.353763,43.882723,45.516546,48.158371,0,270,500
860,52.193048,55.236580,52.753494,52.353464,48.158371,0,328,250


In [17]:
# 3. Random Forest
print("\n=== Random Forest ===")

rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=6,
    random_state=42
)

rf_model.fit(X_train_rf, y_train)

# RF 성능 평가
rf_pred_train = rf_model.predict(X_train_rf)
rf_pred_test = rf_model.predict(X_test_rf)

print("\nRandom Forest 성능:")
print("훈련 세트 성능:")
print(f"RMSE: {np.sqrt(mean_squared_error(y_train, rf_pred_train)):.4f}")
print(f"R2 Score: {r2_score(y_train, rf_pred_train):.4f}")
print("\n테스트 세트 성능:")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, rf_pred_test)):.4f}")
print(f"R2 Score: {r2_score(y_test, rf_pred_test):.4f}")



=== Random Forest ===

Random Forest 성능:
훈련 세트 성능:
RMSE: 15.9076
R2 Score: 0.5494

테스트 세트 성능:
RMSE: 18.5474
R2 Score: 0.3439


In [18]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from category_encoders import TargetEncoder

# 1. 데이터 준비
feature_columns = [
    'distributor', 'category', 'packaging', 'zero_type', 'sweetener_mix',  # 범주형
    'ad_collab', 'price_100ml', 'volume'  # 연속형
]
categorical_features = [
    'distributor', 'category', 'packaging', 'zero_type', 'sweetener_mix'
]
target_column = 'popularity'

X = df[feature_columns]
y = df[target_column]

# 2. 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 3. Random Forest
print("\n=== Random Forest ===")

# Target Encoding
te = TargetEncoder()
X_train_rf = X_train.copy()
X_test_rf = X_test.copy()
X_train_rf[categorical_features] = te.fit_transform(X_train[categorical_features], y_train)
X_test_rf[categorical_features] = te.transform(X_test[categorical_features])

rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=6,
    random_state=42
)

rf_model.fit(X_train_rf, y_train)

# RF 성능 평가
rf_pred_train = rf_model.predict(X_train_rf)
rf_pred_test = rf_model.predict(X_test_rf)

print("\nRandom Forest 성능:")
print("훈련 세트 성능:")
print(f"RMSE: {np.sqrt(mean_squared_error(y_train, rf_pred_train)):.4f}")
print(f"R2 Score: {r2_score(y_train, rf_pred_train):.4f}")
print("\n테스트 세트 성능:")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, rf_pred_test)):.4f}")
print(f"R2 Score: {r2_score(y_test, rf_pred_test):.4f}")

# 4. LightGBM
print("\n=== LightGBM ===")

# Label Encoding for LightGBM
X_train_lgb = X_train.copy()
X_test_lgb = X_test.copy()

from sklearn.preprocessing import LabelEncoder
le_dict = {}
for col in categorical_features:
    le_dict[col] = LabelEncoder()
    X_train_lgb[col] = le_dict[col].fit_transform(X_train_lgb[col])
    X_test_lgb[col] = le_dict[col].transform(X_test_lgb[col])

# LightGBM 모델 정의 - categorical_feature 수정
categorical_indices = [X_train_lgb.columns.get_loc(col) for col in categorical_features]

lgb_model = LGBMRegressor(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42
)

# categorical_feature를 fit 메서드에서 지정
lgb_model.fit(
    X_train_lgb,
    y_train,
    categorical_feature=categorical_indices
)

# LightGBM 성능 평가
lgb_pred_train = lgb_model.predict(X_train_lgb)
lgb_pred_test = lgb_model.predict(X_test_lgb)

print("\nLightGBM 성능:")
print("훈련 세트 성능:")
print(f"RMSE: {np.sqrt(mean_squared_error(y_train, lgb_pred_train)):.4f}")
print(f"R2 Score: {r2_score(y_train, lgb_pred_train):.4f}")
print("\n테스트 세트 성능:")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, lgb_pred_test)):.4f}")
print(f"R2 Score: {r2_score(y_test, lgb_pred_test):.4f}")

# 5. 특성 중요도 비교
# Random Forest 특성 중요도
rf_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

# LightGBM 특성 중요도
lgb_importance = pd.DataFrame({
    'feature': feature_columns,
    'importance': lgb_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n=== Random Forest 특성 중요도 ===")
print(rf_importance)
print("\n=== LightGBM 특성 중요도 ===")
print(lgb_importance)


=== Random Forest ===

Random Forest 성능:
훈련 세트 성능:
RMSE: 15.9076
R2 Score: 0.5494

테스트 세트 성능:
RMSE: 18.5474
R2 Score: 0.3439

=== LightGBM ===
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000154 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 344
[LightGBM] [Info] Number of data points in the train set: 928, number of used features: 8
[LightGBM] [Info] Start training from score 46.710061
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Li

|| CatBoost | random forest | LightGBM |
|-|-|-|-|
|RMSE|18.5163 | 18.5474 | 18.2220|
|R2 Score| 0.3461 | 0.3439 | 0.3668|

- 세 모델중 LightGBM의 예측 오차평균이 가장 낮고 데이터 설명력이 높으므로 해당 알고리즘으로 심층적인 테스트 진행.
- 다음으로 LightGBM모델의 하이퍼파라미터 튜닝 진행.

In [19]:
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder

# 1. 데이터 준비
feature_columns = [
   'distributor', 'category', 'packaging', 'zero_type', 'sweetener_mix',  # 범주형
   'ad_collab', 'price_100ml', 'volume'  # 연속형
]
categorical_features = [
   'distributor', 'category', 'packaging', 'zero_type', 'sweetener_mix'
]
target_column = 'popularity'

X = df[feature_columns]
y = df[target_column]

# 2. 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.2, random_state=42
)

# 3. Label Encoding
X_train_lgb = X_train.copy()
X_test_lgb = X_test.copy()

le_dict = {}
for col in categorical_features:
   le_dict[col] = LabelEncoder()
   X_train_lgb[col] = le_dict[col].fit_transform(X_train_lgb[col])
   X_test_lgb[col] = le_dict[col].transform(X_test_lgb[col])

# categorical_feature를 컬럼 인덱스로 변경
categorical_indices = [X_train_lgb.columns.get_loc(col) for col in categorical_features]

# 4. 그리드 서치를 위한 파라미터 정의
param_grid = {
   'n_estimators': [100, 200, 300],
   'max_depth': [4, 6, 8],
   'learning_rate': [0.01, 0.05, 0.1],
   'num_leaves': [15, 31, 63],
}

# 5. 기본 모델 정의
base_model = LGBMRegressor(
   random_state=42,
   subsample=0.8,
   colsample_bytree=0.8,
   min_child_samples=20
)

# 6. GridSearchCV 수행
grid_search = GridSearchCV(
   estimator=base_model,
   param_grid=param_grid,
   cv=5,
   scoring='neg_root_mean_squared_error',
   n_jobs=-1,
   verbose=2
)

# categorical_feature를 fit 메서드에서 지정
grid_search.fit(X_train_lgb, y_train)

# 7. 최적 파라미터 출력
print("\n최적 파라미터:")
print(grid_search.best_params_)

# 8. 최적 파라미터로 새로운 모델 학습
best_params = grid_search.best_params_
best_model = LGBMRegressor(
   **best_params,
   random_state=42,
   subsample=0.8,
   colsample_bytree=0.8,
   min_child_samples=20
)

best_model.fit(
   X_train_lgb,
   y_train,
   categorical_feature=categorical_indices
)

# 9. 성능 평가
train_pred = best_model.predict(X_train_lgb)
test_pred = best_model.predict(X_test_lgb)

print("\n최적 모델 성능:")
print("훈련 세트 성능:")
print(f"RMSE: {np.sqrt(mean_squared_error(y_train, train_pred)):.4f}")
print(f"R2 Score: {r2_score(y_train, train_pred):.4f}")
print("\n테스트 세트 성능:")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, test_pred)):.4f}")
print(f"R2 Score: {r2_score(y_test, test_pred):.4f}")

# 10. 특성 중요도
feature_importance = pd.DataFrame({
   'feature': feature_columns,
   'importance': best_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\n=== 특성 중요도 ===")
print(feature_importance)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000132 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 341
[LightGBM] [Info] Number of data points in the train set: 928, number of used features: 8
[LightGBM] [Info] Start training from score 46.710061
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM]

- 파라미터 튜닝결과 성능향상이 보이지 않는다.

- 학습 데이터에 대한 예측은 잘 이루어지나 테스트데이터에 대한 낮은 예측률로 보아 모델이 과적합을 일으킬 가능성이 존재한다.
- 과적합을 줄일 수 있는 LightGBM의 bagging모델 사용.

In [20]:
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.base import BaseEstimator, RegressorMixin

# 데이터 로드 및 전처리 (예시)
# df는 주어진 데이터프레임을 가정합니다.

# 1. 범주형 변수 One-Hot Encoding 및 연속형 변수 스케일링
X = df.drop(columns=['popularity'])  # 'popularity'를 종속 변수로 설정
y = df['popularity']  # 타겟 변수는 'popularity'

# 범주형 변수 및 연속형 변수 목록
categorical_features = ['distributor', 'category', 'packaging', 'zero_type', 'sweetener_mix']
numerical_features = ['ad_collab', 'price_100ml', 'volume']

# 전처리 파이프라인 구성
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),  # 연속형 변수 정규화
        ('cat', OneHotEncoder(), categorical_features)  # 범주형 변수 원-핫 인코딩
    ])

# 2. LightGBM 모델을 Wrapping한 클래스를 base_estimator로 사용
class LGBMWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, **params):
        self.model = lgb.LGBMRegressor(**params)

    def fit(self, X, y):
        self.model.fit(X, y)
        return self

    def predict(self, X):
        return self.model.predict(X)

# 3. BaggingRegressor에 LightGBM 모델을 사용
model = LGBMWrapper(num_leaves=31, max_depth=7, learning_rate=0.05)

# BaggingRegressor에 base_estimator로 사용
bagging_model = BaggingRegressor(estimator=model, n_estimators=50, random_state=42)

# 4. 파이프라인 구성
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('bagging', bagging_model)
])

# 5. 훈련 및 테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 6. 모델 훈련
pipeline.fit(X_train, y_train)

# 7. 예측
y_pred_train = pipeline.predict(X_train)
y_pred_test = pipeline.predict(X_test)

# 8. 성능 평가
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
r2_train = r2_score(y_train, y_pred_train)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
r2_test = r2_score(y_test, y_pred_test)

# 결과 출력
print(f"훈련 세트 RMSE: {rmse_train:.4f}")
print(f"훈련 세트 R2: {r2_train:.4f}")
print(f"테스트 세트 RMSE: {rmse_test:.4f}")
print(f"테스트 세트 R2: {r2_test:.4f}")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000190 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 380
[LightGBM] [Info] Number of data points in the train set: 928, number of used features: 40
[LightGBM] [Info] Start training from score 48.511652
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000051 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 373
[LightGBM] [Info] Number of data points in the train set: 928, number of used features: 39
[LightGBM] [Info] Start training from score 47.052490
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000052 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


- 미약한 성능향상이 보인다.

|| LightGBM | LightGBM bagging |
|-|-|-|
|RMSE|18.2220 | 18.5474 |
|R2 Score| 0.3668 |  0.3771 |

- 특정 변수의 문제(예. 과도한 편향성, 또는 이상치)의 문제 일 수 있으므로 변수 분포 재확인.
- LightGBM모델의 변수 중요도 확인.


||feature | importance|
|-|-|-|
|1위| price_100ml  |     2777|
|2위|  volume     |   1617|
|3위|   distributor    |     456|


100ml당 가격과 용량의 중요도가 타 변수보다 과도하게 높은점에 착안하여 일부 변수를 제거하고 과적합을 방지시도.

In [21]:
df.describe()

,ad_collab,price_100ml,volume,popularity
count,1161.000000,1161.000000,1161.000000,1161.000000
mean,0.403962,312.901809,443.474591,46.504633
std,0.490902,195.924833,392.140584,23.553922
min,0.000000,53.000000,75.000000,0.000000
25%,0.000000,174.000000,200.000000,30.861535
50%,0.000000,253.000000,300.000000,46.263781
75%,1.000000,398.000000,500.000000,62.208756
max,1.000000,1793.000000,3000.000000,100.000000


In [22]:
X = df.drop(columns=['popularity'])
y = df['popularity']

categorical_features = ['distributor', 'category', 'packaging', 'zero_type', 'sweetener_mix']
numerical_features = ['ad_collab', 'price_100ml'] # volume 빠짐

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# 2. LightGBM 모델을 Wrapping한 클래스를 base_estimator로 사용
class LGBMWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, **params):
        self.model = lgb.LGBMRegressor(**params)

    def fit(self, X, y):
        self.model.fit(X, y)
        return self

    def predict(self, X):
        return self.model.predict(X)


model = LGBMWrapper(num_leaves=31, max_depth=7, learning_rate=0.05)


bagging_model = BaggingRegressor(estimator=model, n_estimators=50, random_state=42)


pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('bagging', bagging_model)
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)

y_pred_train = pipeline.predict(X_train)
y_pred_test = pipeline.predict(X_test)

rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
r2_train = r2_score(y_train, y_pred_train)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
r2_test = r2_score(y_test, y_pred_test)

# 결과 출력
print(f"훈련 세트 RMSE: {rmse_train:.4f}")
print(f"훈련 세트 R2: {r2_train:.4f}")
print(f"테스트 세트 RMSE: {rmse_test:.4f}")
print(f"테스트 세트 R2: {r2_test:.4f}")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000176 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 331
[LightGBM] [Info] Number of data points in the train set: 928, number of used features: 39
[LightGBM] [Info] Start training from score 48.511652
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000061 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 327
[LightGBM] [Info] Number of data points in the train set: 928, number of used features: 38
[LightGBM] [Info] Start training from score 47.052490
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000051 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, y

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


훈련 세트 RMSE: 13.0799
훈련 세트 R2: 0.6954
테스트 세트 RMSE: 17.7533
테스트 세트 R2: 0.3989


In [23]:
import joblib

# 모델 파이프라인 저장
joblib.dump(pipeline, 'beverage_model_pipeline.pkl')

# 예측 결과 및 평가 지표 저장
results = {
    'Metric': ['RMSE (Train)', 'R2 (Train)', 'RMSE (Test)', 'R2 (Test)'],
    'Value': [rmse_train, r2_train, rmse_test, r2_test]
}
results_df = pd.DataFrame(results)
results_df.to_csv('model_evaluation_metrics.csv', index=False)

# 예측 결과 저장
predictions_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred_test})
predictions_df.to_csv('model_predictions.csv', index=False)

print("모델 및 결과가 저장되었습니다.")


모델 및 결과가 저장되었습니다.


- 테스트 결과 'volume'변수 제외시 성능 향상을 보임.
- 성능향상

|| LightGBM 1st trial |LightGBM 모든변수 | LightGBM '용량' 제외 |
|-|-|-|-|
|RMSE|18.2220 |18.5474  | **17.7533** |
|R2 Score| 0.3668 |0.3771 |  **0.3989** |

- 모델 변수 : pipeline
- 모델 설명력 : 약 40%
- 평균 예측 오차: 17% (100%기준)

모델을 최대한 긍정적으로 설명하자면, 현재까지의 성과는 상당히 의미 있는 수준에 도달했다고 볼 수 있습니다.

### 1. **모델 설명력 40%**:
   - 모델의 **설명력**이 40%라는 것은, 데이터에서 **약 40%의 변동성**을 잘 설명할 수 있다는 것을 의미합니다. 이는 여전히 개선의 여지가 있지만, 여러 모델을 테스트한 결과에 비추어 보면 상당히 괜찮은 수준일 수 있습니다.
   - 특히 복잡한 문제일수록, 설명력이 40%라는 것은 매우 잘한 결과일 수 있습니다. 예를 들어, **복잡한 패턴을 가진 데이터**에서는 40%의 설명력만으로도 **모델이 중요한 패턴을 포착**하고 있다는 신호일 수 있습니다.

### 2. **평균 예측 오차 17% (100% 기준)**:
   - **예측 오차가 17%**라는 것은 예측 값이 실제 값과 비교했을 때, **평균적으로 17%의 차이**를 보인다는 뜻입니다.
   - 100% 기준으로 예측 오차가 17%라는 것은 **상당히 효율적**이라고 할 수 있습니다. 예측의 정확도는 이미 **상당히 높은 수준**에 있고, 데이터에 대한 예측력이 **실용적인 범위**에 도달했다고 볼 수 있습니다.
   - 여러 방법을 테스트하여 **이 정도 성과**를 얻었다는 것은 모델이 상당히 **정교화되었고, 신뢰할 수 있는 예측 결과를 제공**하고 있다는 점에서 긍정적입니다.

### 긍정적인 해석:
- **모델 성능을 개선할 여지가 있다**는 점에서 **향후 추가 개선이 가능**하다는 것을 의미합니다. 예를 들어, 모델의 파라미터 조정이나, 특성 엔지니어링, 더 다양한 모델을 도입하는 방식으로 **설명력과 예측 정확도를 높여갈 수 있는 기회**가 존재합니다.
- **현재의 결과가 실용적인 수준에 도달했다**는 점에서, 이를 실제 환경에 적용하기에 충분한 수준입니다. 예측 오차가 17%라면, **예측의 정확도**는 이미 **실용적인 범위**에 들어가고, 이 정도 정확도가 **비즈니스에서 유용한 결정 지원**을 할 수 있는 수준에 가까워졌습니다.

따라서 현재 모델은 **긍정적인 결과를 도출해냈으며**, 추가적인 튜닝이나 개선을 통해 더 큰 성과를 이룰 가능성이 충분히 존재합니다.

#### 학습/훈련세트 분할 교차검증

In [ ]:
import lightgbm as lgb
import pandas as pd
from sklearn.model_selection import train_test_split, cross_validate, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.base import BaseEstimator, RegressorMixin
import numpy as np

# 데이터 로드 및 전처리 (예시)
# df는 주어진 데이터프레임을 가정합니다.

# 1. 범주형 변수 One-Hot Encoding 및 연속형 변수 스케일링
X = df.drop(columns=['popularity'])  # 'popularity'를 종속 변수로 설정
y = df['popularity']  # 타겟 변수는 'popularity'

# 범주형 변수 및 연속형 변수 목록
categorical_features = ['distributor', 'category', 'packaging', 'zero_type', 'sweetener_mix']
numerical_features = ['ad_collab', 'price_100ml']

# 전처리 파이프라인 구성
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),  # 연속형 변수 정규화
        ('cat', OneHotEncoder(), categorical_features)  # 범주형 변수 원-핫 인코딩
    ])

# 2. LightGBM 모델을 Wrapping한 클래스를 base_estimator로 사용
class LGBMWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, **params):
        self.model = lgb.LGBMRegressor(**params)

    def fit(self, X, y):
        self.model.fit(X, y)
        return self

    def predict(self, X):
        return self.model.predict(X)

# 3. BaggingRegressor에 LightGBM 모델을 사용
model = LGBMWrapper(num_leaves=31, max_depth=7, learning_rate=0.05)

# BaggingRegressor에 estimator로 사용
bagging_model = BaggingRegressor(estimator=model, n_estimators=50, random_state=42)

# 4. 파이프라인 구성
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('bagging', bagging_model)
])

# 5. 교차검증 설정: KFold 교차검증
cv = KFold(n_splits=5, shuffle=True, random_state=42)  # 5-fold 교차검증

# 6. 교차검증 수행
cross_val_results = cross_validate(pipeline, X, y, cv=cv, scoring='neg_mean_squared_error', return_train_score=True)

# 7. 각 폴드별 RMSE 계산 및 출력
print("각 폴드별 RMSE:")
for fold_idx, test_score in enumerate(cross_val_results['test_score']):
    print(test_score)
    rmse = np.sqrt(-test_score)  # 부호 반전 후 제곱근 계산
    print(f"폴드 {fold_idx+1} - RMSE: {rmse:.4f}")


- 교차검증에서 의미있는 성능 향상을 보이진 않는다.

# 새로운 데이터에 대한 예측

In [30]:
df

,distributor,category,packaging,ad_collab,price_100ml,volume,zero_type,popularity,sweetener_mix
0,광동제약,차,페트,1,322,200,일반 음료,69.654006,대체당 없음
1,광동제약,차,페트,1,361,200,일반 음료,43.246780,대체당 없음
2,광동제약,차,유리병,0,528,150,일반 음료,73.306406,대체당 없음
3,광동제약,차,페트,0,273,500,일반 음료,40.011834,대체당 없음
4,광동제약,차,페트,0,296,500,일반 음료,58.161078,대체당 없음


In [31]:
new_df = df.head()
new_df.drop(columns=['popularity'], inplace=True)
new_df

C:\Users\user\AppData\Local\Temp\ipykernel_15992\3758250729.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.drop(columns=['popularity'], inplace=True)


,distributor,category,packaging,ad_collab,price_100ml,volume,zero_type,sweetener_mix
0,광동제약,차,페트,1,322,200,일반 음료,대체당 없음
1,광동제약,차,페트,1,361,200,일반 음료,대체당 없음
2,광동제약,차,유리병,0,528,150,일반 음료,대체당 없음
3,광동제약,차,페트,0,273,500,일반 음료,대체당 없음
4,광동제약,차,페트,0,296,500,일반 음료,대체당 없음


NameError: name 'BaseEstimator' is not defined

In [32]:
import joblib
import pandas as pd
import lightgbm as lgb
from sklearn.base import BaseEstimator, RegressorMixin

class LGBMWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, **params):
        self.model = lgb.LGBMRegressor(**params)

    def fit(self, X, y):
        self.model.fit(X, y)
        return self

    def predict(self, X):
        return self.model.predict(X)


# 저장된 모델 로드
pipeline = joblib.load('beverage_model_pipeline.pkl')


''' 새로운 데이터 : new_df'''

# 모델에 맞게 입력 데이터 가져오기
X_new = new_df[['distributor', 'category', 'packaging', 'zero_type', 'sweetener_mix', 'ad_collab', 'price_100ml']]


# 새로운 데이터 예측
y_new_pred = pipeline.predict(X_new)
y_new_pred
new_df['popularity'] = y_new_pred
new_df



c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


,distributor,category,packaging,ad_collab,price_100ml,volume,zero_type,sweetener_mix,popularity
0,광동제약,차,페트,1,322,200,일반 음료,대체당 없음,45.622759
1,광동제약,차,페트,1,361,200,일반 음료,대체당 없음,39.431742
2,광동제약,차,유리병,0,528,150,일반 음료,대체당 없음,57.974219
3,광동제약,차,페트,0,273,500,일반 음료,대체당 없음,46.095683
4,광동제약,차,페트,0,296,500,일반 음료,대체당 없음,49.032151
